<a href="https://colab.research.google.com/github/brandon-setegn/data-mining-business-analytics/blob/master/west-roxbury-home-values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#let's import the libraries we are going to need
import os, csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

#%matplotlib notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.chdir('/content/drive/MyDrive/Development/machine_learning/dmba')

In [6]:
# Load Data
housing_df = pd.read_csv('WestRoxbury.csv')

In [7]:
housing_df

,TOTAL VALUE,TAX,LOT SQFT,YR BUILT,GROSS AREA,LIVING AREA,FLOORS,ROOMS,BEDROOMS,FULL BATH,HALF BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,4330,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,5190,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,4152,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,6272,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None
4,331.5,4170,5000,1910,2370,1438,2.0,7,3,2,0,1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,404.8,5092,6762,1938,2594,1714,2.0,9,3,2,1,1,1,Recent
5798,407.9,5131,9408,1950,2414,1333,2.0,6,3,1,1,1,1,None
5799,406.5,5113,7198,1987,2480,1674,2.0,7,3,1,1,1,1,None
5800,308.7,3883,6890,1946,2000,1000,1.0,5,2,1,0,1,0,None


In [8]:
housing_df.shape
housing_df.head()
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5802 entries, 0 to 5801
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TOTAL VALUE   5802 non-null   float64
 1   TAX           5802 non-null   int64  
 2   LOT SQFT      5802 non-null   int64  
 3   YR BUILT      5802 non-null   int64  
 4   GROSS AREA    5802 non-null   int64  
 5   LIVING AREA   5802 non-null   int64  
 6   FLOORS        5802 non-null   float64
 7   ROOMS         5802 non-null   int64  
 8   BEDROOMS      5802 non-null   int64  
 9   FULL BATH     5802 non-null   int64  
 10  HALF BATH     5802 non-null   int64  
 11  KITCHEN       5802 non-null   int64  
 12  FIREPLACE     5802 non-null   int64  
 13  REMODEL       5802 non-null   object 
dtypes: float64(2), int64(11), object(1)
memory usage: 634.7+ KB


In [10]:
# prompt: Using dataframe housing_df: find max total value by lot sqft, bucketing lot sqft into groups of 300

housing_df.groupby(pd.cut(housing_df['LOT SQFT '], 300)).agg({'TOTAL VALUE ': 'max'})


,TOTAL VALUE
LOT SQFT,
"(951.586, 1148.38]",176.9
"(1148.38, 1299.76]",295.6
"(1299.76, 1451.14]",244.2
"(1451.14, 1602.52]",NaN
"(1602.52, 1753.9]",213.4
...,...
"(45654.1, 45805.48]",NaN
"(45805.48, 45956.86]",NaN
"(45956.86, 46108.24]",NaN


In [15]:
# Rename columns: replace spaces with '_' to allow dot notation
housing_df = housing_df.rename(columns={'TOTAL VALUE ': 'TOTAL_VALUE'})
housing_df.columns = [s.strip().replace(' ', '_') for s in housing_df.columns] #all columns

In [21]:
# Practice showing the first four rows of the data
housing_df.loc[0:3] # loc[a:b] gives rows a to b, inclusive

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,4330,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,5190,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,4152,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,6272,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None


In [23]:
housing_df.iloc[0:3] # iloc[a:b] gives rows a to b, exclusive (a to b-1)

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,4330,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,5190,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,4152,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None


**Sampling from a Database**

In [31]:
# random sample of 5 observations
housing_df.sample(5)

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
5450,345.0,4340,6035,1950,2520,1240,1.0,6,3,1,1,1,0,Recent
1397,342.1,4303,6518,1930,2359,1434,2.0,6,3,1,0,1,1,None
3621,338.2,4254,5000,1933,3041,1310,1.0,8,3,1,1,1,1,None
418,300.1,3775,5608,1963,2311,1125,1.0,5,3,1,0,1,0,None
4685,452.0,5686,5500,1915,4504,2627,2.0,8,4,1,0,1,1,None


In [30]:
# oversample houses with over 10 rooms
weights = [0.9 if rooms > 10 else 0.01 for rooms in housing_df.ROOMS]
housing_df.sample(5, weights=weights)

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
118,420.8,5293,7861,1967,4182,2888,1.0,12,4,2,1,1,0,None
3164,873.0,10982,19630,1910,6565,3374,2.0,12,4,3,1,1,4,Recent
4329,698.1,8782,6800,1908,5031,3461,2.0,11,6,3,1,1,1,Recent
1866,402.9,5068,7500,1920,4140,2226,1.5,7,2,1,0,1,1,Recent
4603,742.4,9339,12600,1884,6224,3298,2.0,11,7,2,1,1,2,Recent


In [38]:
# REMODEL needs to be converted to a categorical variable
housing_df.REMODEL = housing_df.REMODEL.astype('category')
print(housing_df.REMODEL.cat.categories)
housing_df.REMODEL.dtype

Index(['None', 'Old', 'Recent'], dtype='object')


CategoricalDtype(categories=['None', 'Old', 'Recent'], ordered=False)

In [42]:
# use drop_first=TRUE to drop the first dummy variable
housing_df = pd.get_dummies(housing_df, prefix_sep='_', drop_first=True)
print(housing_df.columns)
housing_df.loc[:, 'REMODEL_Old':'REMODEL_Recent'].head(5)

Index(['TOTAL_VALUE', 'TAX', 'LOT_SQFT', 'YR_BUILT', 'GROSS_AREA',
       'LIVING_AREA', 'FLOORS', 'ROOMS', 'BEDROOMS', 'FULL_BATH', 'HALF_BATH',
       'KITCHEN', 'FIREPLACE', 'REMODEL_Old', 'REMODEL_Recent'],
      dtype='object')


,REMODEL_Old,REMODEL_Recent
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0


# Missing Data

In [43]:
# To illustrate missing data procedures, we first convert a few entries for bedrooms to NA's.  Then we impute these missing values using the median of the ramining values.
missingRows = housing_df.sample(10)
housing_df.loc[missingRows.index, 'BEDROOMS'] = np.nan
housing_df.loc[missingRows.index, 'BEDROOMS']

5180   NaN
2322   NaN
171    NaN
4616   NaN
1055   NaN
1193   NaN
261    NaN
3318   NaN
4611   NaN
2782   NaN
Name: BEDROOMS, dtype: float64

In [45]:
# Remove rows with missing values
reduced_df = housing_df.dropna()
print('Number of rows after removing rows with missing values: ', len(reduced_df))

Number of rows after removing rows with missing values:  5792


In [47]:
# Replace the missing values using the median of the remaining values
medianBedrooms = housing_df.BEDROOMS.median()
housing_df.BEDROOMS = housing_df.BEDROOMS.fillna(medianBedrooms)
print('Number of rows with valid BEDROOMS vlaues after filling NA values: ', housing_df['BEDROOMS'].count())


Numbero f rows with valid BEDROOMS vlaues after filling NA values:  5802


# Normalizing and Rescaling Data

In [48]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
df = housing_df.copy()
df.head()

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
0,344.2,4330,9965,1880,2436,1352,2.0,6,3.0,1,1,1,0,0,0
1,412.6,5190,6590,1945,3108,1976,2.0,10,4.0,2,1,1,0,0,1
2,330.1,4152,7500,1890,2294,1371,2.0,8,4.0,1,1,1,0,0,0
3,498.6,6272,13773,1957,5032,2608,1.0,9,5.0,1,1,1,1,0,0
4,331.5,4170,5000,1910,2370,1438,2.0,7,3.0,2,0,1,0,0,0


In [49]:
# Normalizing Data
# pandas:
norm_df = (housing_df - housing_df.mean()) / housing_df.std()
norm_df.head()

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
0,-0.488879,-0.488507,1.381019,-1.576690,-0.552998,-0.564458,0.710905,-0.691980,-0.271441,-0.568528,0.723202,-0.124803,-1.309337,-0.33356,-0.421381
1,0.200795,0.200789,0.116835,0.229372,0.207196,0.590121,0.710905,2.090325,0.912696,1.347035,0.723202,-0.124803,-1.309337,-0.33356,2.372738
2,-0.631048,-0.631176,0.457697,-1.298834,-0.713635,-0.529303,0.710905,0.699173,0.912696,-0.568528,0.723202,-0.124803,-1.309337,-0.33356,-0.421381
3,1.067927,1.068020,2.807392,0.562799,2.383704,1.759502,-1.536872,1.394749,2.096834,-0.568528,0.723202,-0.124803,0.460235,-0.33356,-0.421381
4,-0.616932,-0.616749,-0.478735,-0.743123,-0.627660,-0.405334,0.710905,0.003597,-0.271441,1.347035,-1.150021,-0.124803,-1.309337,-0.33356,-0.421381


In [51]:
# scikit-learn
scaler = StandardScaler()
# the result of the transformation is a numpy array, we convert it to a datafram
norm_df = pd.DataFrame(scaler.fit_transform(housing_df), index=housing_df.index, columns=housing_df.columns)
norm_df.head()

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
0,-0.488921,-0.488550,1.381138,-1.576825,-0.553046,-0.564507,0.710966,-0.692039,-0.271465,-0.568577,0.723264,-0.124814,-1.309450,-0.333589,-0.421418
1,0.200812,0.200806,0.116845,0.229392,0.207214,0.590172,0.710966,2.090505,0.912775,1.347151,0.723264,-0.124814,-1.309450,-0.333589,2.372943
2,-0.631102,-0.631230,0.457736,-1.298946,-0.713696,-0.529349,0.710966,0.699233,0.912775,-0.568577,0.723264,-0.124814,-1.309450,-0.333589,-0.421418
3,1.068020,1.068112,2.807634,0.562847,2.383909,1.759654,-1.537005,1.394869,2.097015,-0.568577,0.723264,-0.124814,0.460275,-0.333589,-0.421418
4,-0.616985,-0.616802,-0.478777,-0.743187,-0.627714,-0.405369,0.710966,0.003597,-0.271465,1.347151,-1.150120,-0.124814,-1.309450,-0.333589,-0.421418


In [52]:
# Rescaling a data frame

# pandas
norm_df = (housing_df - housing_df.min()) / (housing_df.max() - housing_df.min())
norm_df.head()

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
0,0.214953,0.215015,0.197472,0.934858,0.220237,0.177220,0.5,0.272727,0.250,0.00,0.333333,0.0,0.00,0.0,0.0
1,0.276420,0.276448,0.123156,0.967181,0.311878,0.307628,0.5,0.636364,0.375,0.25,0.333333,0.0,0.00,0.0,1.0
2,0.202283,0.202300,0.143194,0.939831,0.200873,0.181191,0.5,0.454545,0.375,0.00,0.333333,0.0,0.00,0.0,0.0
3,0.353702,0.353740,0.281323,0.973148,0.574253,0.439707,0.0,0.545455,0.500,0.00,0.333333,0.0,0.25,0.0,0.0
4,0.203541,0.203586,0.088145,0.949776,0.211237,0.195193,0.5,0.363636,0.250,0.25,0.000000,0.0,0.00,0.0,0.0


In [53]:
# scikit-learn:
scaler = MinMaxScaler()
norm_df = pd.DataFrame(scaler.fit_transform(housing_df), index=housing_df.index, columns=housing_df.columns)
norm_df.head()

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
0,0.214953,0.215015,0.197472,0.934858,0.220237,0.177220,0.5,0.272727,0.250,0.00,0.333333,0.0,0.00,0.0,0.0
1,0.276420,0.276448,0.123156,0.967181,0.311878,0.307628,0.5,0.636364,0.375,0.25,0.333333,0.0,0.00,0.0,1.0
2,0.202283,0.202300,0.143194,0.939831,0.200873,0.181191,0.5,0.454545,0.375,0.00,0.333333,0.0,0.00,0.0,0.0
3,0.353702,0.353740,0.281323,0.973148,0.574253,0.439707,0.0,0.545455,0.500,0.00,0.333333,0.0,0.25,0.0,0.0
4,0.203541,0.203586,0.088145,0.949776,0.211237,0.195193,0.5,0.363636,0.250,0.25,0.000000,0.0,0.00,0.0,0.0
